Imports

In [14]:
import math
import requests
import pymongo
import time
from secret import *
from global_variables import *

client = mongo_connect(ALGO_TRADING_DB_STRING)
database = get_database(client, 'algo_trading')
basic_data_collection = get_collection(database, 'basic_data')


Gets market summary from polygon api given dates
input: 
    Dates: a list of dates in isoformat (required)
outputs:
    data_list: a list containg dictionaries with the following structure:
        Date: date the data was pulled from
        Data: a dictionary filled with a seperate dictionary for each ticker with the following structure:
            {stock symbol}:
                v: volume (float)
                vw: volume weighted price (float)
                o: open price (float)
                c: close price (float)
                l: low price (float)
                h: high price (float)
                n: number of transactions (int)
                t: unix milliseconds of close time (long)

In [2]:
def get_stock_data(dates):
    data_list = []
    for i in range(len(dates)):
        date = dates[i]
        daily_data = requests.get(f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?adjusted=true&apiKey={POLYGON_API_KEY}')
        daily_data = daily_data.json()
        if daily_data['status'] == 'ERROR': 
            time.sleep(65)
            i -= 1
            continue
        if daily_data['queryCount'] == 0: continue
        daily_data_results = daily_data['results']
        daily_data_dictionary = {}
        for stock in daily_data_results:
            ticker = stock['T']
            stock.pop('T')
            daily_data_dictionary[ticker] = stock
        data_list.append({'Date': date, 'Data': daily_data_dictionary})
        print(f'percent done: {(i / len(dates)) * 100}')
    return data_list

Adds a key to a dictionary and returns the dictionary
inputs:
    key: the key to add to the dictionary (required)
    value: the value to add to the given key (required)
    dic: the dictionary to add the key/value pair to (optional, will add the key to an empty dictionary by default)
output:
    dic: the given dictionary with the key/value pair added

In [3]:
def add_key(key, value, dic = {}):
    dic[key] = value
    return dic

Replaces documents in a mongodb collection that have the same date
inputs:
    collection: the collection to replace documents in (required)
    new_docs: a cursor object containing the documents to replace with (required
    many: a boolean (optional, default false), false means each document in new_docs will replace at most one document in collection with     the same date. True means all documents in collection sharing a date with a document in new_docs will be replace (many -> one)
output: none

In [4]:
def replace_by_date(collection, new_docs, many = False):
    if many == False:
        for doc in new_docs:
            query_filter = {'Date': doc['Date']}
            collection.replace_one(query_filter, doc)
    else:
        for doc in new_docs:
            query_filter = {'Date': doc['Date']}
            collection.delete_many(query_filter)
            collection.insert_one(doc)

Add the percent change to a document containg daily stock data
inputs:

In [5]:
def add_percent_change(collection, documents):
    prev_data = documents.next()['Data']
    for i in range(2):
        curr_doc = documents.next()
        date = curr_doc['Date']
        data = curr_doc['Data']
        for ticker in data.keys():
            try:
                prev_price = prev_data[ticker]['c']
            except KeyError:
                continue
            curr_price = data[ticker]['c']
            price_change_percent = ((curr_price / prev_price) - 1) * 100
            data[ticker]['percent_change'] = price_change_percent
        update_query = {'Date': date}
        update_operator = {'$set':
                {'Data': data}
        }
        collection.update_one(update_query, update_operator)
        prev_data = data
        